In [2]:
import requests 
import sys
sys.path.insert(0, "..")
%load_ext autoreload
%autoreload 2
from src.secrets import API_KEY

In [8]:
import pandas as pd

In [10]:
df = pd.read_csv("../src/state/stops_3city_checked.csv", index_col=0)

In [11]:
df.head()

,stopName,stopDesc,zoneId,zoneName,stopLat,stopLon,coord
0,Dąbrowa Centrum,Gdynia Dąbrowa Centrum,5.0,Gdynia,54.47317,18.46509,"54.473169999999996, 18.46509"
1,Kameliowa,Gdynia Kameliowa,5.0,Gdynia,54.47018,18.46820,"54.470180000000006, 18.4682"
2,Paprykowa,Gdynia Paprykowa,5.0,Gdynia,54.46784,18.46546,"54.46784, 18.46546"
3,Szafranowa,Gdynia Szafranowa,5.0,Gdynia,54.46536,18.46020,"54.46536, 18.4602"
4,Giełda Towarowa,Gdynia Giełda Towarowa,5.0,Gdynia,54.46224,18.45337,"54.46224, 18.45337"


In [16]:
numbers = [i for i in range(0,1100, 100)]
numbers

[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

In [18]:
[(i, i+100) for i in range(0, 1000, 100)]

[(0, 100),
 (100, 200),
 (200, 300),
 (300, 400),
 (400, 500),
 (500, 600),
 (600, 700),
 (700, 800),
 (800, 900),
 (900, 1000)]

In [19]:
from src.data_processing import split_df

In [20]:
dfs = [split_df(df, i, i+100) for i in range(0, 1000, 100)]

In [23]:
origins = df[df['stopDesc'] == 'Niedźwiednik'].iloc[0]
origins_coord = f"{origins.stopLat}, {origins.stopLon}"
origins_coord

'54.38, 18.55937'

In [24]:
from datetime import datetime
time = datetime.now()

In [25]:
from src.distance_matrix_api import get_time_of_travel

In [29]:
results = [get_time_of_travel(origins_coord, dest['coord'].to_list(), time) for dest in dfs]

In [31]:
travel_times = [[i['duration']['value'] for i in result['rows'][0]['elements']] for result in results]

In [33]:
dfs_times = [df.assign(time_sec=time_t) for df, time_t in zip(dfs, travel_times)]

In [35]:
def times_to_colors(t):
    if t < 900:
        color = "green"
    elif 900 <= t < 1800:
        color = "blue"
    elif 1800 <= t < 2700:
        color = "yellow"
    elif 2700 <= t < 3600:
        color = "red"
    else:
        color = "black"
    return color

In [36]:
df_colors = [df.assign(color=df.time_sec.apply(lambda x: times_to_colors(x))) for df in dfs_times]

In [37]:
df_colors[0]

,stopName,stopDesc,zoneId,zoneName,stopLat,stopLon,coord,time_sec,color
45,Św. Wojciech,Święty Wojciech,1.0,Gdańsk,54.28478,18.63808,"54.28478000000001, 18.63808",3548,red
49,Olimpijska,Olimpijska,1.0,Gdańsk,54.30558,18.58112,"54.305580000000006, 18.58112",3637,black
23,Maćkowy,Maćkowy,1.0,Gdańsk,54.30689,18.61525,"54.30689, 18.61525",3540,red
29,Przegalina,Przegalina,1.0,Gdańsk,54.31122,18.91707,"54.31122, 18.91707",4920,black
48,Kampinoska,Kampinoska,1.0,Gdańsk,54.31611,18.60301,"54.31611, 18.60301",3060,red
...,...,...,...,...,...,...,...,...,...
3,Szafranowa,Gdynia Szafranowa,5.0,Gdynia,54.46536,18.46020,"54.46536, 18.4602",3300,red
2,Paprykowa,Gdynia Paprykowa,5.0,Gdynia,54.46784,18.46546,"54.46784, 18.46546",3240,red
1,Kameliowa,Gdynia Kameliowa,5.0,Gdynia,54.47018,18.46820,"54.470180000000006, 18.4682",3232,red
0,Dąbrowa Centrum,Gdynia Dąbrowa Centrum,5.0,Gdynia,54.47317,18.46509,"54.47317, 18.46509",3120,red


In [38]:
df_all = pd.DataFrame()
for df in df_colors:
    df_all = df_all.append(df)

In [39]:
df_all

,stopName,stopDesc,zoneId,zoneName,stopLat,stopLon,coord,time_sec,color
45,Św. Wojciech,Święty Wojciech,1.0,Gdańsk,54.28478,18.63808,"54.28478000000001, 18.63808",3548,red
49,Olimpijska,Olimpijska,1.0,Gdańsk,54.30558,18.58112,"54.305580000000006, 18.58112",3637,black
23,Maćkowy,Maćkowy,1.0,Gdańsk,54.30689,18.61525,"54.30689, 18.61525",3540,red
29,Przegalina,Przegalina,1.0,Gdańsk,54.31122,18.91707,"54.31122, 18.91707",4920,black
48,Kampinoska,Kampinoska,1.0,Gdańsk,54.31611,18.60301,"54.31611, 18.60301",3060,red
...,...,...,...,...,...,...,...,...,...
988,NaN,Energetyków,NaN,Gdynia,54.53967,18.49937,"54.53967, 18.499370000000006",3960,black
910,NaN,Chylonia Krzywoustego,NaN,Gdynia,54.54594,18.46845,"54.54594, 18.46845",4168,black
911,NaN,Bosmańska - Nasypowa,NaN,Gdynia,54.54853,18.52011,"54.54853000000001, 18.52011",4348,black
912,NaN,Elektrociepłownia - Kontenerowa,NaN,Gdynia,54.55106,18.48669,"54.55106, 18.48669",4860,black


In [42]:
lat_stops = [float(i) for i in df_all['stopLat'].to_list()]
lon_stops = [float(i) for i in df_all['stopLon'].to_list()]

In [40]:
origin_lat = float(origins_coord.split(', ')[0])
origin_lon = float(origins_coord.split(', ')[1])

In [45]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap

In [46]:
output_file("gmap_all.html")

map_options = GMapOptions(lat=origin_lat, lng=origin_lon, map_type="roadmap", zoom=11)

p = gmap(API_KEY, map_options, title="3city")

source = ColumnDataSource(
                    data=dict(lat=lat_stops,
                              lon=lon_stops,
                              color=df_all.color.to_list())
)
origin = ColumnDataSource(
                    data=dict(lat=[origin_lat],
                              lon=[origin_lon])
)

p.circle(x="lon", y="lat", size=15, fill_color="color", fill_alpha=0.8, source=source)
p.circle(x="lon", y="lat", size=20, fill_color="grey", fill_alpha=0.8, source=origin)

show(p)